In [2]:
import open3d as o3d
import numpy as np
import sympy as sp
from open3d.web_visualizer import draw
import modern_robotics as mr

In [3]:
def Ry_sym(theta):
    ct = sp.cos(theta)
    st = sp.sin(theta)
    R = sp.Matrix([[ct, 0.0, st], [0.0, 1.0, 0.0], [-st, 0, ct]])
    return R

def Rx_sym(theta):
    ct = sp.cos(theta)
    st = sp.sin(theta)
    R = sp.Matrix([[1.0, 0.0, 0.0], [0.0, ct, -st], [0.0, st, ct]])
    return R

def skew(v):
    return sp.Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def exp6(twist, theta):
    omega = skew(twist[:3])
    v = sp.Matrix(twist[3:])
    T = sp.eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (sp.eye(3) * theta + (1 - sp.cos(theta)) * omega +
              (theta-sp.sin(theta)) * omega * omega) * v
    return T

In [4]:
def calc_v(omega_mat, q_mat):
    #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    assert len(omega_mat) == len(q_mat)
    
    n_joints = omega_mat.shape[1] 
    v_mat = sp.zeros(3, n_joints)      

    for i in range(n_joints):
        v_mat[:,i] = (-skew(omega_mat.col(i)) * q_mat.col(i))
    return v_mat

def Slist_maker(omega_mat, q_mat): #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    v_mat = calc_v(omega_mat, q_mat)    
    n_joints = omega_mat.shape[1]
    Slist = sp.zeros(6, n_joints)
    
    for i in range(n_joints):
        Slist[:3,i] = omega_mat[:,i]
        Slist[3:,i] = v_mat[:,i]
    return Slist


In [5]:
M1=sp.Matrix([[0, 1, 0, 0],
             [1, 0, 0, 0],
             [0, 0, -1, 200],
             [0, 0, 0, 1]])

M2=sp.Matrix([[1, 0, 0, 25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])

M3=sp.Matrix([[-1, 0, 0, 455+25],
             [0, 0, 1, 0],
             [0, 1, 0, 400],
             [0, 0, 0, 1]])

M4=sp.Matrix([[0, 0, -1, 455+25+420],
             [0, 1, 0, 0],
             [1, 0, 0, 400],
             [0, 0, 0, 1]])
Mlist = np.array([M1,M2,M3,M4], dtype=float)

S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])

S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
Slist[:, :4]

om = sp.zeros(3,4)
om1 = om[:, 0] = sp.Matrix([0,0,-1])
om2 = om[:, 1] = sp.Matrix([0,1,0])
om3 = om[:, 2] = sp.Matrix([0,1,0])
om4 = om[:, 3] = sp.Matrix([-1,0,0])
q = sp.zeros(3,4)
q1 = q[:,0] = M1[:3, 3]
q2 = q[:,1] = M2[:3, 3]
q3 = q[:,2] = M3[:3, 3]
q4 = q[:,3] = M4[:3, 3]

S_list = Slist_maker(om,q)
S_list

Matrix([
[ 0,    0,    0,   -1],
[ 0,    1,    1,    0],
[-1,    0,    0,    0],
[ 0, -400, -400,    0],
[ 0,    0,    0, -400],
[ 0,   25,  480,    0]])

In [6]:
class RobotObject:
    robotObjects = [o3d.geometry.TriangleMesh.create_coordinate_frame(size=75)] #List of all o3d objects, initialized with {s}-frame

    def __init__(self, Mlist, link_orient = 'z'):
        self.current_config = Mlist 
        self.Mlist = Mlist
        self.num_joints = len(Mlist)
        self.num_links = len(Mlist)
        self.length_links = self.findLinkLengths(Mlist)
        self.link_orient = ['z']*(self.num_links+1) if link_orient == 'z' else link_orient

        self.joints = []
        self.links = []
        self.make_robot_objects() # create all objects of robot (links, frames, joints)
        self.__transform(Mlist) #Transforms all objects from {s} to zero-config

#Calculates link lengths based on M
    def findLinkLengths(self, Mlist):
        linkLengths = np.zeros(self.num_links)
        
        for i in range(self.num_links):
            p = Mlist[i][:3,3]
            p_pre = np.array([0,0,0]) if i==0 else Mlist[i-1][:3,3] 
            linkLengths[i] = np.linalg.norm(p_pre-p)
        return linkLengths

#creates all o3d-objects of the robot in {s}
    def make_robot_objects(self): 
        for i in range(self.num_joints):
            self.joints.append(Joint())
        for i in range(self.num_links):
            self.links.append(Link(self.length_links[i], self.link_orient[i]))

    def allToOrigin(self): #Sends all objects to Origin
        T_origin = []
        for T in self.current_config:
            T_origin.append(mr.TransInv(T))
        self.__transform(T_origin)
        return

    def transform(self, Slist, thetas):
        self.allToOrigin() 
        T_list = []  # List with T01,T02,T03...
        T = np.eye(4)
        # for i in range(len(thetas)):
        #     Smat = mr.VecTose3(Slist[:, i])
        #     T = T @ mr.MatrixExp6(Smat * thetas[i])
        #     T_list.append(T@Mlist[i])
        for i in range(len(thetas)):
            T = T @ exp6(Slist[:, i], thetas[i])
            T_list.append(T@Mlist[i])   
        self.__transform(T_list)
        return


#Moves all objects from {s} to config given by T_list
    def __transform(self, T_list): #Private member function
        for i, J in enumerate(self.joints):
            J.transform(T_list[i])
        for i, L in enumerate(self.links):
            L.transform(T_list[i])

    def draw_robot(self):
        draw(self.robotObjects)

#_____________Joint Class______________#
class Joint(RobotObject):

    def __init__(self):
        self.joint = o3d.geometry.TriangleMesh.create_cylinder(
            radius=10, height=30)
        self.coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.joint)
        self.robotObjects.append(self.coord)

    def set_colour(self, colour=[0, 1, 0]):
        self.joint.paint_uniform_color(colour)

    def transform(self, T):
        self.joint = self.joint.transform(T)
        self.coord = self.coord.transform(T)

#____________Link Class_________________#
class Link(RobotObject):

    def __init__(self, lenght, orient):
        self.lenght = lenght
        
        if (orient == 'x'): ##Defines link direction
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Ry_sym(np.pi/2)).translate(np.array([self.lenght/2, 0, 0]))
        elif (orient == 'y'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Rx_sym(-np.pi/2)).translate(np.array([0, self.lenght/2, 0]))
        elif (orient == 'z'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).translate(np.array([0, 0, self.lenght/2]))
        
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.link)

    def set_colour(self, colour=[0, 0, 1]):
        self.link.paint_uniform_color(colour)

    def transform(self, T):
        self.link = self.link.transform(T)

In [13]:

KUK = RobotObject( Mlist, ['z', 'y', 'x', 'z'])
# KUK2 = RobotObject( Mlist[:2], ['z', 'y', 'x'])
thetas = [-1.1416, -1.3258,  1.3102, -1.5708]

# KUK.theta_transform(Slist, [-1.1416, -1.3258,  1.3102, -1.5708])
# KUK2.theta_transform(Slist[:,:2], [pi, pi/2])



# KUK.transform(S_list, [0,np.pi/4,np.pi/4,0])
KUK.draw_robot()


WebVisualizer(window_uid='window_6')

In [8]:
M4

Matrix([
[0, 0, -1, 900],
[0, 1,  0,   0],
[1, 0,  0, 400],
[0, 0,  0,   1]])

In [9]:
thetas = [0,0,0,0]
T_list = []  # List with T01,T02,T03...
T = np.eye(4)
for i in range(len(thetas)):
    T = T @ exp6(Slist[:, i], thetas[i])
    T_list.append(T@Mlist[i])
T_list


[Matrix([
 [  0, 1.0,    0,     0],
 [1.0,   0,    0,     0],
 [  0,   0, -1.0, 200.0],
 [  0,   0,    0,   1.0]]),
 Matrix([
 [1.0,    0,   0,  25.0],
 [  0,    0, 1.0,     0],
 [  0, -1.0,   0, 400.0],
 [  0,    0,   0,   1.0]]),
 Matrix([
 [-1.0,   0,   0, 480.0],
 [   0,   0, 1.0,     0],
 [   0, 1.0,   0, 400.0],
 [   0,   0,   0,   1.0]]),
 Matrix([
 [  0,   0, -1.0, 900.0],
 [  0, 1.0,    0,     0],
 [1.0,   0,    0, 400.0],
 [  0,   0,    0,   1.0]])]

In [10]:
thetas = [np.pi,0,0,np.pi/10]

T_list = []  # List with T01,T02,T03...

T = np.eye(4)
T1 = T @ mr.MatrixExp6(mr.VecTose3(S1) * thetas[0])
T2 = T1 @ mr.MatrixExp6(mr.VecTose3(S2) * thetas[1])
T3 = T2 @ mr.MatrixExp6(mr.VecTose3(S3) * thetas[2])
T4 = T3 @ mr.MatrixExp6(mr.VecTose3(S4) * thetas[3])


In [11]:
f0 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
f1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
f2 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
f3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
f4 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)

draw([f1,f2,f3,f4])
f2.transform(M1)
draw([f1,f2,f3,f4])
f3.transform(M2)
draw([f1,f2,f3,f4])
f4.transform(M4)
draw([f0,f1,f2,f3,f4])

WebVisualizer(window_uid='window_1')

WebVisualizer(window_uid='window_2')

WebVisualizer(window_uid='window_3')

WebVisualizer(window_uid='window_4')